# Generating tiles from raw TIFF files

This notebook downloads the TIFF files obtained from Google Earth Engine, preprocesses the images into 500x500 (for features) or 50x50 (for labels) tiles for training and testing, and saves everything into Numpy files.


## 0. Install libraries and imports

In [1]:
# !pip -q install huggingface_hub rasterio affine tqdm 

In [2]:
import re, json
from pathlib import Path
from huggingface_hub import snapshot_download
import numpy as np
import rasterio
from rasterio import windows
from rasterio.warp import reproject
from rasterio.enums import Resampling
from affine import Affine
from tqdm import tqdm
from numpy.lib.format import open_memmap

## 1. Configure settings

### Declare constants for processing

In [3]:
# Working directory to download TIFF files to and processed tiles. 
# This will require a few 100GB of space so uh make sure you do this on a 1 TB external drive or something
WORK_DIR  = Path("E:\\RSI09\\v4")
# Make some directories to process files
DATASET_DIR   = WORK_DIR / "tiff"
TILES_DIR = WORK_DIR / "tiles"

for p in [DATASET_DIR, TILES_DIR]:
    p.mkdir(parents=True, exist_ok=True)
    
# To prevent significant class imbalance, we only train on tiles that have minimum number of pixels identified as flood.
# This threshold is the same used in MMFlood
MIN_FLOOD_PCT = 2.0   

# Size of tiles
PRED_TILE = (500, 500)  # predictor tile size (50 m * 10)
LBL_TILE  = (50, 50)    # label tile size (500 m / 10)

# Filenames to identify which raw tiffs belong to the test dataset
TEST_FILENAMES = ["99.1717616_17.1286267", "4.7049263_9.9129092"]

# List of features in the dataset
# Make sure "label" is the last item
FEATURES = ["dem", "slope", "ndvi", "lc", "water", "label"]

## 2. Download and group files

### Download data if necessary

In [4]:
# Download data from HuggingFace
# snapshot_download(repo_id="MatthiasWang/geotiffs", repo_type="dataset", local_dir=DATASET_DIR)

### Group files belonging to a single flood event together

In [5]:
# Google Earth Engine will split a single export into multiple files if the export is too large
# We need to keep track of all these split files
# The filenames are in the following format:
# {folder}_flood_<lon>_<lat>-0000-0000...

# RegEx for the filename
BASE_KEY_RE = re.compile(
    r"^[a-z]+_flood_(-?\d+(?:\.\d+)?)_(-?\d+(?:\.\d+)?)(?:-\d+)*$",
    flags=re.IGNORECASE
)

# Extract base key "<lon>_<lat>" from "{folder}_flood_<lon>_<lat>-0000-0000..." (stem)
def extract_base_key(stem):
    m = BASE_KEY_RE.match(stem)
    
    if m: 
        # Filename is in the format "{folder}_flood_<lon>_<lat>-0000-0000..."
        return f"{m.group(1)}_{m.group(2)}"
    
    # Filename is in the format "{folder}_flood_<lon>_<lat>"
    m2 = re.match(r"^[a-z]+_flood_(.+)$", stem, flags=re.IGNORECASE)
    return m2.group(1) if m2 else None

# Next, we need a dictionary to keep track of all the floods, for each flood event, for each feature
# i.e., we need a dictionary like so:
'''
maps: {
    "dem": {
        -98.7136208_49.7307341: [dem_flood_-98.7136208_49.7307341-0000-0000, dem_flood_-98.7136208_49.7307341-0000-0032, ...],
        -93.8267856_30.2350467: [dem_flood_-93.8267856_30.2350467-0000-0000, dem_flood_-93.8267856_30.2350467-0000-0032, ...],
        ...
    }

    "slope": {
        -98.7136208_49.7307341: [slope_flood_-98.7136208_49.7307341-0000-0000, slope_flood_-98.7136208_49.7307341-0000-0032, ...],
        -93.8267856_30.2350467: [slope_flood_-93.8267856_30.2350467-0000-0000, slope_flood_-93.8267856_30.2350467-0000-0032, ...],
        ...
    }

    ...
} 
'''

# Valid file extensions
VALID_EXT = {".tif", ".tiff"}

def list_folder_files_grouped(folder):
    out = {}

    # This shouldn't happen, but just in case
    if not folder.exists(): return out

    for p in folder.iterdir():
        if not p.is_file(): continue

        if p.suffix.lower() not in VALID_EXT and "." in p.name:
            pass

        base = extract_base_key(p.stem)

        # If extract_base_key returns nothing
        if not base: continue

        out.setdefault(base, []).append(p)
    return out

folders = {k: DATASET_DIR / k for k in FEATURES}
maps = {k: list_folder_files_grouped(v) for k, v in folders.items()}

# Keep only locations that exist in ALL folders
# Keys is an array storing "<lon>_<lat>" for all valid locations
keys = set.intersection(*(set(m.keys()) for m in maps.values())) if all(maps.values()) else set()
print(f"Found {len(keys)} locations with complete rasters (grouped).")

Found 8 locations with complete rasters (grouped).


## 3. Extract tiles from maps

### Helper functions to reproject onto fixed tile sizes

In [6]:
def reproject_many_into_tile(paths, dst_transform, dst_crs, dst_shape, *, categorical):
    """
    Reproject multiple source rasters into a single destination tile (dst_shape).
    Later rasters do NOT erase existing valid values with NaN:
      - We reproject each source into a temp tile, then merge where temp is finite and dst is NaN.
    This keeps memory use small and avoids OOM.
    """
    H, W = dst_shape
    dst = np.full((H, W), np.nan, dtype=np.float32)
    for p in paths:
        try:
            with rasterio.open(p) as src:
                temp = np.full((H, W), np.nan, dtype=np.float32)
                reproject(
                    source=rasterio.band(src, 1),
                    destination=temp,
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=dst_transform,
                    dst_crs=dst_crs,
                    resampling=(Resampling.nearest if categorical else Resampling.bilinear),
                    dst_nodata=np.nan,
                    num_threads=2
                )
                # Merge: fill only where dst is NaN and temp is finite
                m = ~np.isnan(temp) & np.isnan(dst)
                if m.any():
                    dst[m] = temp[m]
        except Exception:
            continue
    return dst

def binarise(arr):
    return (arr > 0).astype(np.uint8)

### Function to extract tiles for training and testing data

In [7]:
def preprocess_all_streaming(keys, maps, tiles_dir):
    tiles_dir.mkdir(parents=True, exist_ok=True)
    kept_total, seen_total = 0, 0
    thr = MIN_FLOOD_PCT / 100.0

    for key in tqdm(sorted(keys), desc="Preprocessing (streaming tiles)"):
        dem_list   = maps["dem"][key]
        slope_list = maps["slope"][key]
        ndvi_list  = maps["ndvi"][key]
        lc_list    = maps["lc"][key]
        water_list = maps["water"][key]
        lbl_list   = maps["label"][key]

        # Process each DEM chunk individually
        for dem_path in dem_list:
            with rasterio.open(dem_path) as dem_src:
                crs = dem_src.crs
                transform = dem_src.transform
                height, width = dem_src.height, dem_src.width

                # Only full 500x500 tiles
                Hc = (height // PRED_TILE[0]) * PRED_TILE[0]
                Wc = (width  // PRED_TILE[1]) * PRED_TILE[1]
                if Hc == 0 or Wc == 0:
                    continue

                for r0 in range(0, Hc, PRED_TILE[0]):
                    for c0 in range(0, Wc, PRED_TILE[1]):
                        seen_total += 1

                        # Use a window to read 500x500 chunks
                        win = windows.Window(c0, r0, *PRED_TILE)
                        tile_transform = windows.transform(win, transform)

                        # DEM
                        dem = dem_src.read(1, window=win, masked=True).filled(np.nan).astype(np.float32)

                        # Label on coarse grid (50×50)
                        coarse_transform = tile_transform * Affine.scale(10, 10)
                        y_coarse = reproject_many_into_tile(lbl_list, coarse_transform, crs, LBL_TILE, categorical=True)
                        y = binarise(y_coarse)

                        # We only enforce minimum flood threshold for training dataset
                        data_split = "test"
                        flood_ratio = float(np.nanmean(y))  # y has 0/1; NaNs rare but guard anyway
                        if key not in TEST_FILENAMES:
                            data_split = "train"
                            if not np.isfinite(flood_ratio) or flood_ratio < thr:
                                continue  # reject  

                        # Slope / NDVI / LC / Water
                        slope = reproject_many_into_tile(slope_list, tile_transform, crs, PRED_TILE, categorical=False)
                        ndvi = reproject_many_into_tile(ndvi_list, tile_transform, crs, PRED_TILE, categorical=False)
                        lc = reproject_many_into_tile(lc_list, tile_transform, crs, PRED_TILE, categorical=True)
                        water = reproject_many_into_tile(water_list, tile_transform, crs, PRED_TILE, categorical=False)

                        # For NDVI, Slope, Water, divide by 1000 to convert back to float32
                        slope /= 1000
                        ndvi /= 1000
                        water /= 1000

                        # Make sure the stacked features are in the same order as FEATURES array
                        x = np.stack([dem, slope, ndvi, lc, water], axis=0).astype(np.float32)
                        x = np.nan_to_num(x, nan=0.0, posinf=0.0, neginf=0.0)

                        meta = {
                            "key": key,
                            "dem_chunk": Path(dem_path).name,
                            "r": int(r0), "c": int(c0),
                            "Hc": int(Hc), "Wc": int(Wc),
                            "predictor_tile_shape": [len(FEATURES) - 1, PRED_TILE[0], PRED_TILE[1]],
                            "label_tile_shape": [LBL_TILE[0], LBL_TILE[1]],
                            "flood_ratio": flood_ratio,
                        }

                        tile_name = f"{key}__{Path(dem_path).stem}__r{r0}c{c0}_{data_split}"
                        np.savez_compressed(tiles_dir / f"{tile_name}.npz",
                                            x=x.astype(np.float32),
                                            y=y.astype(np.uint8),
                                            meta=json.dumps(meta))
                        kept_total += 1

    print(f"Saved {kept_total} / {seen_total} tiles "
          f"(kept {100.0 * kept_total / seen_total:.1f}%)"
          f"to {tiles_dir}")

In [8]:
preprocess_all_streaming(list(keys), maps, TILES_DIR)

Preprocessing (streaming tiles): 100%|██████████| 8/8 [4:01:36<00:00, 1812.01s/it]  

Saved 7811 / 115759 tiles (kept 6.7%)to E:\RSI09\v4\tiles


## 4. Save training and testing dataset to Numpy files

### Load the metadata for each tile

In [9]:
def _decode_meta(meta_raw):
    # meta was saved as JSON string; handle various npz dtypes robustly
    try:
        if hasattr(meta_raw, "dtype") and meta_raw.dtype.kind in "OUS":
            return json.loads(meta_raw.item())
        try:
            return json.loads(meta_raw.tobytes().decode("utf-8"))
        except Exception:
            return json.loads(str(meta_raw))
    except Exception:
        return {}


### Function to stack tiles into a single Numpy file

In [10]:
# tile_paths is a list of .npz files
# out_base is the stem of the filepath to save the stacked npz files to e.g /kaggle/working/flood_stack
def save_numpy_stack_from_tiles(tile_paths, out_base):
    # Define paths
    N = len(tile_paths)
    X_path = f"{out_base}_X.npy"
    Y_path = f"{out_base}_Y.npy"
    meta_path = f"{out_base}_meta.jsonl"
    stats_path = f"{out_base}_stats.json"

    # Create the output npy files
    X_mm = open_memmap(X_path, mode="w+", dtype=np.float32, shape=(N, len(FEATURES) - 1, PRED_TILE[0], PRED_TILE[1]))
    Y_mm = open_memmap(Y_path, mode="w+", dtype=np.uint8,   shape=(N, LBL_TILE[0], LBL_TILE[1]))

    # Used to compute normalization/standardization stats (mean, variance, etc.)
    sums = np.zeros(len(FEATURES) - 1, dtype=np.float64)
    sumsq = np.zeros(len(FEATURES) - 1, dtype=np.float64)
    counts = np.zeros(len(FEATURES) - 1, dtype=np.int64)

    with open(meta_path, "w") as fmeta:
        for i, p in enumerate(tqdm(tile_paths, desc="Collating tiles to NumPy")):
            rec = np.load(p, allow_pickle=False)
            # Feature maps
            x = rec["x"]
            # Label
            y = rec["y"]
            m = _decode_meta(rec["meta"])
            m["predictor_tile_shape"] = [len(FEATURES) - 1, PRED_TILE[0], PRED_TILE[1]]

            X_mm[i] = x.astype(np.float32)
            Y_mm[i] = y.astype(np.uint8)

            for c in range(len(FEATURES) - 1):
                feature = x[c]
                finite = np.isfinite(feature)

                # No NaN values
                if finite.any():
                    vals = feature[finite].astype(np.float64)
                    sums[c]  += vals.sum()
                    sumsq[c] += (vals * vals).sum()
                    counts[c] += vals.size

            fmeta.write(json.dumps(m) + "\n")

    del X_mm, Y_mm  # flush

    means = (sums / np.maximum(counts, 1)).tolist()
    variances = (sumsq / np.maximum(counts, 1) - np.square(sums / np.maximum(counts, 1)))
    stds = np.sqrt(np.maximum(variances, 0)).tolist()

    with open(stats_path, "w") as f:
        json.dump({
            "N": N,
            "channels": FEATURES[:-1],
            "mean": means,
            "std": stds,
            "counts": counts.tolist()
        }, f, indent=2)

    return {"X": X_path, "Y": Y_path, "meta": meta_path, "stats": stats_path, "N": N}


### Save train

In [11]:
# All paths where tiles are stored
full_tile_paths = sorted(Path(TILES_DIR).glob("*.npz"))

In [12]:
print("Saving training data...")
train_paths = []
for path in full_tile_paths:
    if "train" in str(path):
        train_paths.append(str(path))

STACK_BASE = WORK_DIR / "train"
stack_info = save_numpy_stack_from_tiles(train_paths, STACK_BASE)
print(stack_info)

Saving training data...


Collating tiles to NumPy: 100%|██████████| 6603/6603 [22:21<00:00,  4.92it/s]


{'X': 'E:\\RSI09\\v4\\train_X.npy', 'Y': 'E:\\RSI09\\v4\\train_Y.npy', 'meta': 'E:\\RSI09\\v4\\train_meta.jsonl', 'stats': 'E:\\RSI09\\v4\\train_stats.json', 'N': 6603}


### Save test

In [13]:
# For testing dataset, we want each testing example to be exported as its own npz file
print("Saving testing data...")
for test_filename in TEST_FILENAMES:
    test_paths = []
    
    for path in full_tile_paths:
        if test_filename in str(path) and "test" in str(path):
            test_paths.append(str(path))

    STACK_BASE = WORK_DIR / f"test_{test_filename}"
    stack_info = save_numpy_stack_from_tiles(test_paths, STACK_BASE)
    print(stack_info)


Saving testing data...


Collating tiles to NumPy: 100%|██████████| 608/608 [01:14<00:00,  8.14it/s]


{'X': 'E:\\RSI09\\v4\\test_99.1717616_17.1286267_X.npy', 'Y': 'E:\\RSI09\\v4\\test_99.1717616_17.1286267_Y.npy', 'meta': 'E:\\RSI09\\v4\\test_99.1717616_17.1286267_meta.jsonl', 'stats': 'E:\\RSI09\\v4\\test_99.1717616_17.1286267_stats.json', 'N': 608}


Collating tiles to NumPy: 100%|██████████| 600/600 [01:10<00:00,  8.50it/s]


{'X': 'E:\\RSI09\\v4\\test_4.7049263_9.9129092_X.npy', 'Y': 'E:\\RSI09\\v4\\test_4.7049263_9.9129092_Y.npy', 'meta': 'E:\\RSI09\\v4\\test_4.7049263_9.9129092_meta.jsonl', 'stats': 'E:\\RSI09\\v4\\test_4.7049263_9.9129092_stats.json', 'N': 600}
